In [1]:
import sys
sys.path.append('../src/')

%load_ext autoreload
%autoreload 2

# Introduction

In this notebook I will show a few examples how you can use the global fishing watch module. The modules are simple wrappers around the [GFW API](https://globalfishingwatch.org/our-apis/documentation#introduction) (version3).

This is all work in progress. 

You will need an API key from Global Fishing Watch and add that as an environmental variable to the repository (for instance in a .env file). Be sure not to publish this key in your code!

All code except (geo)pandas is available in the standard Python library.


In [41]:
import gfw
import pandas as pd
from pathlib import Path
import geopandas as gpd

In [40]:
PATH_DATA = Path.cwd().parent.joinpath('data')

## Get vessels

With a basic search you can query the vessel database (which contains more than 100.000 vessels). You can use MMSI or IMO numbers. The IMO number is persistent during the lifetime of a vessel. The MMSI can change when ownership or flag is changed, so it's semi-persistent. GFW uses MMSI as the principal identifier of a vessel. An IMO number will often result in multiple vessels being returned. An MMSI query can also return multiple vessels, but only because the data is from different sources. Usually you can deduplicate the resulting dataframe or keep the duplicate rows with the most information. 

In [9]:
# Search with MMSI

list_of_vessels = [538005957, 636019558, 511100921]

vessels, owners = gfw.get_vessels(query=list_of_vessels,
                           filename='vessels', 
                           field='mmsi')

print(f'Found {len(vessels)} vessels and {len(owners)} owners')

Found 4 vessels and 1 owners


In [6]:
# Let's check out the vessels

vessels

,id,sourceCode,ssvid,flag,shipname,nShipname,callsign,imo,latestVesselInfo,transmissionDateFrom,transmissionDateTo,geartypes,lengthM,tonnageGt,vesselInfoReference,messagesCounter,positionsCounter,matchFields
0,509dd770de5aa17235d569f255f24fcd,[IMO],538005957,MHL,HALITYILDIRIM,HALITYILDIRIM,V7IK9,9257981,True,2014-10-01T12:12:36Z,2023-11-30T23:58:55Z,[CARGO],NaN,17979.0,4169a368-df89-48c1-b034-273015c678ef,NaN,NaN,NaN
1,5cf28fc7f-fa71-0102-79c9-eae9e5a7cd11,[AIS],538005957,MHL,None,None,None,None,NaN,2015-02-21T16:50:15Z,2023-05-08T03:46:34Z,NaN,NaN,NaN,NaN,159.0,52.0,NO_MATCH
2,db2efbda8-876a-255f-8fec-5dcb34f1ec4a,[AIS],636019558,LBR,PETREL S,PETRELS,D5VB2,9363883,NaN,2020-02-03T13:31:59Z,2024-02-09T23:59:53Z,NaN,NaN,NaN,NaN,72643681.0,673203.0,NO_MATCH
3,75549a637-7d39-7a59-b367-ea3167026168,[AIS],511100921,PLW,HELGA,HELGA,T8A4144,9419151,NaN,2023-02-06T10:29:44Z,2024-02-09T23:58:35Z,NaN,NaN,NaN,NaN,24019961.0,424892.0,NO_MATCH


In [7]:
# Let's check out the owners

owners

,name,flag,ssvid,sourceCode,dateFrom,dateTo
0,YIL SHIPPING,MHL,538005957,[IMO],2014-10-01T12:12:36Z,2023-11-30T23:58:55Z


In [ ]:
# Write to file

vessels.to_csv(PATH_DATA.joinpath('vessels_found.csv'), index=False)

## Get events

There are multiple events identified in the GFW database:
- loitering: a ship remains stationary outside the coast
- encounters: two or more vessels are in close proximity to each other during a certain amount of time
- port visits
- fishing
- ais gaps: moments that the AIS has been turned off, or the signal has dropped

You can query the events by vessel id (from the GFW database), so if you don't have this id, you need to use get_events first. Or you can query the events by geometry. That is a bit more tricky, because you know how to work with geospatial data. If you need a geometry, [geojson.io](https://geojson.io/) is a good place to get started. The geometry function does a POST request to Global Fishing Watch. Make sure the coordinate reference system is WGS84 (latitudes, longitudes), also [known as EPSG:4326](https://epsg.io/4326). If you use geojson.io you have the right coordinate reference system.

In [36]:
# Get loitering events

vessel = vessels.id[1]

loitering = gfw.get_events(vessel_id=vessel,
                           event_type='loitering',
                           filename='loitering',
                           start_date='2012-01-01',
                           end_date='2024-01-01')

print(f'Found {len(loitering)} loitering events')

Found 1 loitering events


In [38]:
# Get port visits

vessel = vessels.id[2]

visits = gfw.get_events(vessel_id=vessel,
                        event_type='port_visits',
                        filename='portvistis',
                        start_date='2012-01-01',
                        end_date='2024-01-01')

print(f'Found {len(visits)} port visits')

Found 180 port visits


In [ ]:
# Or if you want to query multiple vessels

# Get unique vessel ids
ids = vessels.id.unique()

# Create empty list
dfs = []

for id in ids:
    df = gfw.get_events(vessel_id=id,
                        event_type='port_visits',
                        filename='port_visits',
                        start_date='2012-01-01',
                        end_date='2024-01-01')

    dfs.append(df)

df = pd.concat(dfs)
df.reset_index(drop=True, inplace=True)
len(df)



In [ ]:
# Write visits to file

visits.to_csv(PATH_DATA.joinpath('port_visits.csv'), index=False)

## Get vessels by geometry

In [52]:
# Import geometry (often it's easy to work with geopandas/geodataframes)

gdf = gpd.read_file(PATH_DATA.joinpath('map.geojson'))

# Get geometry in right shape

geometry = gdf.geometry.__geo_interface__

In [54]:
loitering = gfw.get_events_by_geometry(start_date='2023-01-01',
                                     end_date='2023-04-01',
                                     event_type='loitering',
                                     filename='loitering_events',
                                     geometry=geometry)

print(f'Found {len(loitering)} loitering events in provided geometry')

Found 925 loitering events in provided geometry
